In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-sales-data/sales_data_sample.csv


In [2]:
filepath = "/kaggle/input/sample-sales-data/sales_data_sample.csv"
df = pd.read_csv(filepath, encoding='ISO-8859-1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [4]:
def simplify_address(row):
    if pd.notnull(row['ADDRESSLINE1']) and pd.notnull(row['ADDRESSLINE2']):
        return f"{row['ADDRESSLINE1']}, {row['ADDRESSLINE1']}"
    else:
        return row['ADDRESSLINE1']

# Apply the function to create a new merged column
df['ADDRESS'] = df.apply(simplify_address, axis=1)
df = df.drop(['ADDRESSLINE1', 'ADDRESSLINE2'], axis=1)

In [5]:
customer_col = ['CUSTOMERNAME', 'PHONE', 'ADDRESS','CONTACTLASTNAME', "CONTACTFIRSTNAME", "CITY", 'STATE', 'POSTALCODE', 'COUNTRY', 'TERRITORY']
product_col = ['PRODUCTCODE', 'PRODUCTLINE', "MSRP"]
date_col = ['ORDERDATE', 'QTR_ID', 'MONTH_ID', 'YEAR_ID']

customer_info = df[customer_col]
product_info = df[product_col]
date_info = df[date_col]

drop_cols = customer_info.columns[1:].to_list() + product_info.columns.to_list() + date_info.columns[1:].to_list()
orders = df.drop(drop_cols, axis=1)
orders.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,CUSTOMERNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,Land of Toys Inc.,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,Reims Collectables,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,Lyon Souveniers,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,Toys4GrownUps.com,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,Corporate Gift Ideas Co.,Medium


# Calendar Lookup
required for time-series analysis with Power BI

In [6]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

# Find the start and end dates
start_date = df['ORDERDATE'].min()
end_date = df['ORDERDATE'].max()

calendar = pd.date_range(start_date, end_date, freq='D')

calendar = pd.DataFrame({'date_id': range(len(calendar)), 'date': calendar})

# Export to CSV
calendar.to_csv('calendar.csv', index=False)

# Create Dimension Tables

In [7]:
def create_dim_table(data, col_id):
    new_df = data.drop_duplicates()
    new_df = new_df.reset_index(drop=True)
    if isinstance(new_df, pd.Series):
        new_df = new_df.to_frame()
    new_df[col_id] = new_df.index
    return new_df

In [8]:
orders['ORDERDATE'] = pd.to_datetime(orders['ORDERDATE'])
orders.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,CUSTOMERNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,Land of Toys Inc.,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,Reims Collectables,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,Lyon Souveniers,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,Toys4GrownUps.com,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,Corporate Gift Ideas Co.,Medium


In [9]:
date = create_dim_table(data=date_info, col_id='date_id')
product = create_dim_table(data=product_info, col_id='product_id')
product_line = create_dim_table(data=product_info["PRODUCTLINE"], col_id='productline_id')

In [10]:
customer = create_dim_table(data=customer_info, col_id="customer_id")
customer.head()

,CUSTOMERNAME,PHONE,ADDRESS,CONTACTLASTNAME,CONTACTFIRSTNAME,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,customer_id
0,Land of Toys Inc.,2125557818,897 Long Airport Avenue,Yu,Kwai,NYC,NY,10022,USA,NaN,0
1,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Henriot,Paul,Reims,NaN,51100,France,EMEA,1
2,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Da Cunha,Daniel,Paris,NaN,75508,France,EMEA,2
3,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Young,Julie,Pasadena,CA,90003,USA,NaN,3
4,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,Brown,Julie,San Francisco,CA,NaN,USA,NaN,4


In [11]:
location = create_dim_table(data=customer[['CITY', 'STATE', 'COUNTRY', 'TERRITORY']], col_id='location_id')
location.head()

,CITY,STATE,COUNTRY,TERRITORY,location_id
0,NYC,NY,USA,NaN,0
1,Reims,NaN,France,EMEA,1
2,Paris,NaN,France,EMEA,2
3,Pasadena,CA,USA,NaN,3
4,San Francisco,CA,USA,NaN,4


In [12]:
city = create_dim_table(data=location['CITY'],col_id='city_id')
state = create_dim_table(data=location['STATE'], col_id='state_id')
territory = create_dim_table(data=location['TERRITORY'], col_id='territory_id')
country = create_dim_table(data=location['COUNTRY'], col_id='country_id')

In [13]:
product['productline_id'] = product['PRODUCTLINE'].map(product_line.set_index('PRODUCTLINE')['productline_id'])

In [14]:
customer['city_id'] = df['CITY'].map(city.set_index('CITY')['city_id'])
customer['state_id'] = df['STATE'].map(state.set_index('STATE')['state_id'])
customer['territory_id'] = df['TERRITORY'].map(territory.set_index('TERRITORY')['territory_id'])
customer['country_id'] = df['COUNTRY'].map(country.set_index('COUNTRY')['country_id'])

In [15]:
location['city_id'] = df['CITY'].map(city.set_index('CITY')['city_id'])
location['state_id'] = df['STATE'].map(state.set_index('STATE')['state_id'])
location['territory_id'] = df['TERRITORY'].map(territory.set_index('TERRITORY')['territory_id'])
location['country_id'] = df['COUNTRY'].map(country.set_index('COUNTRY')['country_id'])

In [16]:
order_status = create_dim_table(data=df['STATUS'], col_id='status_id')
order_status.head()

,STATUS,status_id
0,Shipped,0
1,Disputed,1
2,In Process,2
3,Cancelled,3
4,On Hold,4


In [17]:
order_size = create_dim_table(data=df['DEALSIZE'], col_id='size_id')
order_size.head()

,DEALSIZE,size_id
0,Small,0
1,Medium,1
2,Large,2


### Remove Redundant Columns

In [18]:
product = product.drop('PRODUCTLINE', axis=1)
product.head()
location = location.drop(['CITY','STATE','COUNTRY','TERRITORY'], axis=1, errors='ignore')
location.head()
customer = customer.drop(['CITY','STATE','COUNTRY','TERRITORY'], axis=1,errors='ignore')
customer.head()

,CUSTOMERNAME,PHONE,ADDRESS,CONTACTLASTNAME,CONTACTFIRSTNAME,POSTALCODE,customer_id,city_id,state_id,territory_id,country_id
0,Land of Toys Inc.,2125557818,897 Long Airport Avenue,Yu,Kwai,10022,0,0,0,0,0
1,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Henriot,Paul,51100,1,1,1,1,1
2,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Da Cunha,Daniel,75508,2,2,1,1,1
3,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Young,Julie,90003,3,3,2,0,0
4,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,Brown,Julie,NaN,4,4,2,0,0


# Create Fact Table

In [19]:
orders['date_id'] = df['ORDERDATE'].map(calendar.set_index('date')['date_id'])

orders['product_id'] = df['PRODUCTCODE'].map(product.set_index('PRODUCTCODE')['product_id'])

orders['customer_id'] = df['CUSTOMERNAME'].map(customer.set_index('CUSTOMERNAME')['customer_id'])

orders['status_id'] = df['STATUS'].map(order_status.set_index('STATUS')['status_id'])

orders['size_id'] = df['DEALSIZE'].map(order_size.set_index('DEALSIZE')['size_id'])

In [20]:
orders = orders.drop(['ORDERDATE','CUSTOMERNAME','STATUS','DEALSIZE'], axis=1, errors='ignore')

# Export to CSV

In [21]:
def save_to_csv(data, filename):
    data.to_csv(filename, header=True, index=None)

In [22]:
save_to_csv(data=orders,filename='orders.csv')
save_to_csv(data=customer,filename='customer.csv')
save_to_csv(data=product,filename='product.csv')
save_to_csv(data=product_line,filename='product_line.csv')
save_to_csv(data=order_status,filename='order_status.csv')
save_to_csv(data=order_size,filename='order_size.csv')
save_to_csv(data=city,filename='city.csv')
save_to_csv(data=country,filename='country.csv')
save_to_csv(data=territory,filename='territory.csv')
save_to_csv(data=state,filename='state.csv')


In [23]:
save_to_csv(data=date, filename='date.csv')

# Export to Postgres

In [24]:
# from sqlalchemy import create_engine

# # Connect to PostgreSQL database
# username = 'postgres'
# password = 'sqlpassword'
# hostname = 'localhost'
# port = 5432
# database_name = 'sales_data'
# connection_str = f'postgresql://{username}:{password}@{hostname}:{port}/{database_name}'
# engine = create_engine(connection_str)

In [25]:
# orders.to_sql('orders', engine, if_exists='replace', index=False)
# customer.to_sql('customer', engine, if_exists='replace', index=False)
# product.to_sql('product', engine, if_exists='replace', index=False)
# product_line.to_sql('product_line', engine, if_exists='replace', index=False)
# order_status.to_sql('order_status', engine, if_exists='replace', index=False)
# order_size.to_sql('order_size', engine, if_exists='replace', index=False)
# city.to_sql('city', engine, if_exists='replace', index=False)
# country.to_sql('country', engine, if_exists='replace', index=False)
# territory.to_sql('territory', engine, if_exists='replace', index=False)
# state.to_sql('state', engine, if_exists='replace', index=False)